<a href="https://colab.research.google.com/github/doelleb/algoverse-jtad/blob/main/RLHF_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install dependencies
!pip install trl datasets accelerate peft transformers --quiet
!pip install bitsandbytes --quiet  # for faster loading

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#load base model
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "sshleifer/tiny-gpt2"  # super small, fast for demo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

In [3]:
#load HH-RLHF dataset
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/hh-rlhf", split="train[:300]")
print(dataset[0])

README.md:   0%|          | 0.00/356 [00:00<?, ?B/s]

train_human_turn0.csv: 0.00B [00:00, ?B/s]

train_human_turn0_sampled.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/44035 [00:00<?, ? examples/s]

{'human_turn0': 'Hi, I want to learn to play horseshoes. Can you teach me?', 'human_turn0_token_count': 12}


In [8]:
#train a simple reward model distilbert-style to predict preference (chosen > rejected)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split

reward_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
reward_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

def preprocess_reward(example):
    return {
        "text": [example["chosen"], example["rejected"]],
        "label": [1, 0]
    }

raw_dataset = dataset.select(range(300))  # select a few examples

reward_data = []
for ex in raw_dataset:
    reward_data.append({"text": ex["chosen"], "label": 1})
    reward_data.append({"text": ex["rejected"], "label": 0})

reward_dataset = Dataset.from_list(reward_data)
reward_dataset = reward_dataset.train_test_split(test_size=0.1)

def tokenize_reward(example):
    return reward_tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_reward = reward_dataset.map(tokenize_reward, batched=True)

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="no",
    num_train_epochs=1,
    output_dir="./reward_model_output",
    logging_dir="./logs",
)

trainer = Trainer(
    model=reward_model,
    args=training_args,
    train_dataset=tokenized_reward["train"],
    eval_dataset=tokenized_reward["test"],
)
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 'chosen'

In [ ]:
#PPO fine-tuning with TRL
from trl import PPOTrainer, PPOConfig
import torch

ppo_config = PPOConfig(
    model_name=model_name,
    learning_rate=1e-5,
    batch_size=1,
    mini_batch_size=1,
    gradient_accumulation_steps=1,
    optimize_cuda_cache=False,
)

ppo_trainer = PPOTrainer(ppo_config, model, tokenizer, reward_model=reward_model)

# Create mini prompt set for demo
prompts = ["What is the capital of France?", "Explain photosynthesis."] * 5

for prompt in prompts:
    query_tensor = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    response_tensor = model.generate(query_tensor, max_new_tokens=30)
    full_input = torch.cat([query_tensor, response_tensor], dim=1)

    with torch.no_grad():
        reward = reward_model(tokenizer.batch_decode(full_input, skip_special_tokens=True)[0], return_dict=True).logits[0]

    reward_score = torch.tensor([reward.item()])
    ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward_score)

In [ ]:
#save models
model.save_pretrained("./ppo_rlhf_baseline")
tokenizer.save_pretrained("./ppo_rlhf_baseline")
reward_model.save_pretrained("./reward_model")